In [ ]:
import json
import sys
from pathlib import Path

import numpy as np

In [ ]:
quality_check_metrics_file = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/siren/experiment_scripts/logs/chair_base_seat_ply/metrics.json")
with open(quality_check_metrics_file, "r") as f:
    quality_check_metrics = json.load(f)

In [ ]:
filled_difference_fractions = []
for value in quality_check_metrics.values():
    filled_difference_fractions.append(value["filled_difference_fraction"])
filled_difference_fractions = np.array(filled_difference_fractions)

In [ ]:
filled_difference_fractions.mean()
np.percentile(filled_difference_fractions, [30, 50, 75])

In [ ]:
quality_check_metrics